# Retroscan with a MISP warninglist

## Introduction

- UUID: **9469c0d5-2d79-4c2e-8727-fa9321411e92**
- Started from [issue 8](https://github.com/MISP/misp-playbooks/issues/8)
- State: **Published** : demo version with **output**
- Purpose: This playbook does a *retroscan* to check for attributes matching the values in a warninglist. You can then disable the to_ids flag or add a tag or comment.
    - This playbook acts as a *retroscan* for threat intelligence **curation** when you add for example a new warninglist to MISP.
    - The results are summarised and sent to Mattermost and TheHive.
    - Also have a look at "[Create a custom MISP warninglist](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_create_custom_MISP_warninglist.ipynb)" for ways to create or edit warninglists via MISP playbooks.
- Tags: [ "warninglist", "hunting", "false-positives", "indicator-quality", "curation" ]
- External resources: **Mattermost**, **TheHive**
- Target audience: **SOC, CSIRT**, **CTI**


# Playbook

- **Retroscan with a MISP warninglist**
    - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Set helper variables
- **Warninglist**
    - WL:1 MISP warninglists
    - WL:2 Set the MISP warninglist name or ID
    - WL:3 Get the warninglist values
- **Retroscan**
    - RS:1 Define the retroscan filters
    - RS:2 Show retroscan matches
    - RS:3 Retroscan actions
    - RS:4 Indicators (attributes) and event list
- **Closure**
    - EN:1 Create the summary of the playbook 
    - EN:2 Send a summary to Mattermost
    - EN:3 Send an alert to TheHive
    - EN:4 End of the playbook 
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Indicate if PyMISP should attempt to verify the certificate or ignore errors
mattermost_playbook_user="<MATTERMOST USER>"
mattermost_hook="<MATTERMOST WEBHOOK>"
thehive_url="<THEHIVE URL>"
thehive_key="<THEHIVE API KEY>"
virustotal_apikey="<VIRUSTOTAL_APIKEY>"
```

In [2]:
# Initialise Python environment
import urllib3
import sys
import json
from pyfaup.faup import Faup
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
import uuid
from uuid import uuid4
from pymisp import *
from pymisp.tools import GenericObjectGenerator
import re
import time

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

The version of PyMISP recommended by the MISP instance (2.4.176) is newer than the one you're using now (2.4.173). Please upgrade PyMISP.


The Python libraries are loaded and the credentials are read from the keys file.
I will use the MISP server https://misp.demo.cudeso.be/ for this playbook.




## PR:2 Set helper variables

This cell contains **helper variables** that are used in this playbook.

In [3]:
# Values from the warninglist
search_values = []

# Matches in MISP
retroscan_matches = []

# Affected indicators (attributes), their ID and UUIDs and the events they are in
affected_indicators = []
affected_indicators_id = []
affected_indicators_uuid = []
affected_indicators_event_id = []

# A set of regular expressions that we use to determine the attribute type in a warninglist
regular_expressions = {"sha256": "^[a-fA-F0-9]{64}$",
                       "md5": "^[a-fA-F0-9]{32}$",
                       "hostname": "^[a-zA-Z0-9.\-_]+\.[a-zA-Z]{2,}$",
                       "sha1": "^[a-fA-F0-9]{40}$",
                       "url": "^(http|https):\/\/[-a-zA-Z0-9-]{2,256}\.[-a-zA-Z0-9-]{2,256}",
                       "ip-src": "[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}",
                       "ip-dst": "[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}"
                      }

# Warninglist

## WL:1 MISP warninglists

The [misp-warninglists](https://github.com/MISP/misp-warninglists) are lists of **well-known** indicators that can be associated to potential **false positives**, errors or mistakes. The warning lists are integrated in MISP to display an info/warning box at the event and attribute level if such indicators are available in one of the list. The lists are also used to filter potential false-positive at API level. MISP warninglists also serve a dual use. You cannot only use them to identify false positives, you can also use the lists to **track** if specific attributes (for example IPs or domains) are in the threat events you receive from your community.

## WL:2 Set the MISP warninglist name or ID

Specify the MISP warninglist in `retroscan_warninglist`. If you supply a numberical value, the playbook attempts to read the warninglist by ID. If you supply a non-numerical value it will lookup the warninglist by name.

In [4]:
# Provide the warninglist ID (numerical value) or the name of the warninglist
retroscan_warninglist = "Camp20"

# Code block to check if a warninglist exists.
warninglist_exist = False
misp_warninglist = False

try:
    retroscan_warninglist = int(retroscan_warninglist)
    if retroscan_warninglist > 0:
        misp_warninglist = misp.get_warninglist(retroscan_warninglist, pythonify=True)
        if not "errors" in misp_warninglist:
            warninglist_exist = True
            print("Found a matching warninglist \033[92m{}\033[90m.".format(misp_warninglist.name))
        else:
            print("Received an error from MISP when looking up warninglist ID \033[91m{}\033[90m.".format(retroscan_warninglist))
    else:
        print("Unable to proceed with a negative warninglist ID \033[91m{}\033[90m.".format(retroscan_warninglist))
except ValueError:
    all_warninglists = misp.warninglists(pythonify=True)
    if len(all_warninglists) > 0:
        for warninglist in all_warninglists:
            if warninglist.name == retroscan_warninglist:
                misp_warninglist = misp.get_warninglist(warninglist.id, pythonify=True)
                warninglist_exist = True
                print("Found a matching warninglist with ID \033[92m{}\033[90m.".format(misp_warninglist.id))
                break

if warninglist_exist:
    if misp_warninglist.enabled and len(misp_warninglist.WarninglistEntry) > 0:
        print(" Warninglist is \033[92menabled\033[90m.")
        print(" Warninglist has \033[92m{}\033[90m entries.".format(len(misp_warninglist.WarninglistEntry)))
        print("\n")
    else:
        if not misp_warninglist.enabled:
            print(" Unable to proceed. Warninglist is \033[91mnot enabled\033[90m.")
        else:
            print(" Unable to proceed. Warninglist has \033[91mno entries\033[90m.")

Found a matching warninglist with ID 96.
 Warninglist is enabled.
 Warninglist has 6 entries.




## WL:3 Get the warninglist values

The next cell processes the warninglist values so that we can search for them in MISP. The warninglist values are returned by PyMISP in a list stored in `WarninglistEntry`.

In [5]:
for value in misp_warninglist.WarninglistEntry:
    if "value" in value:
        search_values.append(value["value"])

# Optionally print the values
#print(search_values)

print("Warninglist values stored in list \033[92msearch_values\033[90m.\n")

Warninglist values stored in list search_values.



# Retroscan

## RS:1 Define the retroscan filters

Define the retroscan **filters**. For all the filters, if you set the variable to `None` then the filter gets ignored.

- Limit the search to **recently changed attributes** with `attribute_timestamp`
- Limit the **number of results** with `limit`
- Only include attributes from **published** events with `published`
- Only match with attributes that have **to_ids** set by using `to_ids`
- Only match with attributes of a **specific type** with the list `limit_attribute_type`
- **Exclude** attributes of a specific type with `ignore_attribute_type`

In [7]:
# Limit to recently changed attributes or include all (None)
attribute_timestamp = None

# No limit in returned matches
limit = 0

# Include published events (1), unpublished (0) or both (None)
published = None  

# Include only matches where attributes have to_ids (1) or not (0) or both (None)
to_ids = None

# Only get these specific attributes
limit_attribute_type =  [] # []

# Ignore specific types
ignore_attribute_type = ["malware-sample"] #["malware-sample"]

# Sanitise
limit = int(limit)
if not('limit_attribute_type' in locals() and isinstance(limit_attribute_type, list)):
    limit_attribute_type = []
if not ('ignore_attribute_type' in locals() and isinstance(ignore_attribute_type, list)):
    ignore_attribute_type = []

## RS:2 Show retroscan matches

The next cell queries MISP with the filters you provided earlier.

In [8]:
print("Searching in MISP")

if limit > 0:
    print(" Limit to \033[92m{}\033[90m results.".format(limit))
print(" Attribute timestamp: \033[92m{}\033[90m | Published: \033[92m{}\033[90m | To_ids:  \033[92m{}\033[90m".format(attribute_timestamp, published, to_ids))
print(" Limit attribute types to \033[92m{}\033[90m".format(limit_attribute_type))
print(" Ignore attribute types \033[91m{}\033[90m".format(ignore_attribute_type))

search_matches = misp.search("attributes", type_attribute=limit_attribute_type, value=search_values, attribute_timestamp=attribute_timestamp, to_ids=to_ids, published=published, limit=limit, include_context=True, pythonify=True)
for match in search_matches:
    if match.type not in ignore_attribute_type:
        retroscan_matches.append(match)

print("Found \033[92m{}\033[90m matches.\n".format(len(retroscan_matches)))

Searching in MISP
 Attribute timestamp: None | Published: None | To_ids:  None
 Limit attribute types to []
 Ignore attribute types ['malware-sample']
Found 5 matches.



### Print the retroscan matches

You can now print the retroscan matches and then in the next section decide which actions you'd like to take.

In [9]:
sort_column = "Warninglist value"

# Put the results in a pretty table. We can use this table later also for the summary
misp_table = PrettyTable()

misp_table.field_names = ["Warninglist value", "Attr. ID", "Event ID", "Event", "Published", "Category", "Type", "IDS", "Comment", "Tags", "Object relation", "Timestamp"]
misp_table.align = "l"
misp_table._max_width = {"Event" : 25, "Comment": 25}

for match in retroscan_matches:
    object_relation = ""
    tags = ""
    for tag in match.tags:
        tags = "{}{} ".format(tags, tag.name)
    if hasattr(match, "object_relation"):
        object_relation = match.object_relation
    misp_table.add_row([match.value, match.id, match.event_id, match.Event.info, match.Event.published, match.category, match.type, match.to_ids, match.comment, tags, object_relation, match.timestamp])
print(misp_table.get_string(sortby=sort_column))

+------------------------------------------------------------------+----------+----------+---------------------------+-----------+------------------+----------+-------+---------+--------------------------------------+-----------------+---------------------------+
| Warninglist value                                                | Attr. ID | Event ID | Event                     | Published | Category         | Type     | IDS   | Comment | Tags                                 | Object relation | Timestamp                 |
+------------------------------------------------------------------+----------+----------+---------------------------+-----------+------------------+----------+-------+---------+--------------------------------------+-----------------+---------------------------+
| d252235aa420b91c38bfeec4f1c3f3434bc853d04635453648b26b2947352889 | 2647415  | 3100     | Malware triage for        | False     | Payload delivery | sha256   | True  |         |                              

## RS:3 Retroscan actions

Now for each result you can do a specific action
- Add one or more **tags** with the list `action_add_tag`. With `action_add_tag_local` you indicate if the tags are local or global. Because the retroscan is often used for **curation** of older results it's advised to use local, as curation tags should remain within your MISP instance.
- **Disable the IDS flag** with `action_disable_ids`
- Add a **comment** to the attribute with `action_add_comment`. This comment will be appended to existing comments.

In general the actions are applied against all the attributes. If you only want to apply them against a subset of the results then add the attribute ID to the list `action_attribute_list`. If you leave the list empty then the actions are applied against **all** attributes.

In [10]:
# List of attribute IDs to apply the actions. Have an empty list for all
action_attribute_list = []

# List of tags to apply
action_add_tag = ["customer:curated=\"disable_ids_retroscan\""]

# Disable the to_ids for matches
action_disable_ids = True

# Local or global tag
action_add_tag_local = True

# Add a comment
action_add_comment = None


# Sanitise
if not('action_add_tag' in locals() and isinstance(action_add_tag, list)):
    action_add_tag = []
if not('action_attribute_list' in locals() and isinstance(action_attribute_list, list)):
    action_attribute_list = []    
print("Retroscan actions")

for match in retroscan_matches:
    if len(action_attribute_list) > 0:
        if match.id not in action_attribute_list:
            print(" Skipping \033[91m{}\033[90m with ID: {}".format(match.value, match.id))
            break
    
    if len(action_add_tag) > 0:
        print(" Add tags to \033[92m{}\033[90m (ID: {}) in event {}".format(match.value, match.id, match.event_id))
        for tag in action_add_tag:
            t = MISPTag()
            t.name = tag
            t.local = action_add_tag_local
            misp.tag(match.uuid, t)
            
    if action_disable_ids:
        print(" Disable IDS of \033[91m{}\033[90m (ID: {}) in event {}".format(match.value, match.id, match.event_id))
        match.to_ids = False
        misp.update_attribute(match)
        
    if action_add_comment:
        print(" Add comment to \033[92m{}\033[90m (ID: {}) in event {}".format(match.value, match.id, match.event_id))        
        match.comment = "{} {}".format(match.comment, action_add_comment.strip())
        misp.update_attribute(match)
        
    if match.value not in affected_indicators:
        affected_indicators.append(match.value)
    if match.id not in affected_indicators_id:
        affected_indicators_id.append(match.id)
    if match.uuid not in affected_indicators_uuid:
        affected_indicators_uuid.append(match.uuid)
    if match.event_id not in affected_indicators_event_id:
        affected_indicators_event_id.append(match.event_id)        
print("Finished retroscan actions")

Retroscan actions
 Add tags to www.domainnotonlist1.com (ID: 2635857) in event 2662
 Disable IDS of www.domainnotonlist1.com (ID: 2635857) in event 2662
 Add tags to f17616ec0522fc5633151f7caa278caa (ID: 2647413) in event 3100
 Disable IDS of f17616ec0522fc5633151f7caa278caa (ID: 2647413) in event 3100
 Add tags to d252235aa420b91c38bfeec4f1c3f3434bc853d04635453648b26b2947352889 (ID: 2647415) in event 3100
 Disable IDS of d252235aa420b91c38bfeec4f1c3f3434bc853d04635453648b26b2947352889 (ID: 2647415) in event 3100
 Add tags to www.outzdemo.be (ID: 2647420) in event 2656
 Disable IDS of www.outzdemo.be (ID: 2647420) in event 2656
 Add tags to www.outzdemo.be (ID: 2647421) in event 2662
 Disable IDS of www.outzdemo.be (ID: 2647421) in event 2662
Finished retroscan actions


## RS:4 Indicators (attributes) and event list

The next cell prints the list of indicators that were affected and the events they were in.

In [11]:
print("Indicators")
for el in affected_indicators:
    print(" {}".format(el))
print("\n\nEvents")
for el in affected_indicators_event_id:
    print(" {}".format(el))
print("\n\n")

Indicators
 www.domainnotonlist1.com
 f17616ec0522fc5633151f7caa278caa
 d252235aa420b91c38bfeec4f1c3f3434bc853d04635453648b26b2947352889
 www.outzdemo.be


Events
 2662
 3100
 2656





# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed and can also be send to a chat channel. 

## EN:1 Create the summary of the playbook 

The next section creates a summary and stores the output in the variable `summary` in Markdown format. It also stores an intro text in the variable `intro`. These variables can later be used when sending information to Mattermost or TheHive.

In [12]:
summary = "# MISP Playbook summary\nRetroscan with a MISP warninglist. \n\n"

summary_warninglist = "## Warninglist: {}\n\n".format(misp_warninglist.name)
summary_warninglist += "- Description: **{}**\n".format(misp_warninglist.description)
summary_warninglist += "- Type: **{}**\n".format(misp_warninglist.type)
misp_warninglist_category = "False positive"
if misp_warninglist.category == "known":
    misp_warninglist_category = "Known identifier"
summary_warninglist += "- Category: **{}**\n".format(misp_warninglist_category)
summary_warninglist += "- Version: **{}**\n".format(misp_warninglist.version)
summary_warninglist += "- Entries: **{}**\n".format(len(misp_warninglist.WarninglistEntry))
summary_warninglist += "\n\n"

summary += summary_warninglist

intro = summary

summary += "## Retroscan \n"
summary += "### Filters\n"
if limit > 0:
    summary += "- Limit to **{}** results\n".format(limit)
summary += "- Attribute timestamp: **{}**\n".format(attribute_timestamp)
summary += "- Published: **{}**\n".format(published)
summary += "- To_ids: **{}**\n".format(to_ids)
summary += "- Limit attribute types to **{}**\n".format(limit_attribute_type)
summary += "- Ignore attribute types **{}**\n".format(ignore_attribute_type)
summary += "\n\n"
summary += "### Values\n"
for value in search_values:
    summary += "{}\n".format(value)
summary += "\n\n"

summary += "### Results\n"
misp_table.set_style(MARKDOWN)
summary += misp_table.get_string(sortby="Warninglist value")
summary += "\n\n"

summary += "### Actions\n"
if len(action_attribute_list) > 0:
    summary += "- Only apply actions against attributes with ID: **{}**\n".format(action_attribute_list)
if len(action_add_tag) > 0:
    summary += "- Add tags: **{}**\n".format(action_add_tag)    
if action_disable_ids:
    summary += "- **Disable IDS**\n"
if action_add_comment:
    summary += "- Add comment: **{}**\n".format(action_add_comment)    
    
summary += "### Indicators\n"
for el in affected_indicators:
    summary += "{}\n".format(el)
summary += "\n\n"

summary += "### Events\n"
for el in affected_indicators_event_id:
    summary += "{}\n".format(el)
summary += "\n\n"

summary += "### Attribute IDs\n"
for el in affected_indicators_id:
    summary += "{}\n".format(el)
summary += "\n\n"

summary += "### Attribute UUID\n"
for el in affected_indicators_uuid:
    summary += "{}\n".format(el)
summary += "\n\n"

print("The \033[92msummary\033[90m of the playbook is available.\n")

The summary of the playbook is available.



## EN:2 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [13]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [14]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

Summary is sent to Mattermost.



## EN:3 Send an alert to TheHive

Next to informing your colleagues via Mattermost you can also send an **alert** to TheHive. The alert contains the summary, and the list matches as 'observables'.

You can change the alert title with `thehive_alert_title` and provide a reference type with `thehive_alert_reference`. Note that this reference needs to be **unique** in TheHive. If you want to create multiple alerts for the same MISP event then add some random value at the end.

In [15]:
# The title of the TheHive alert
thehive_alert_title = "MISP Playbook Summary - Retroscan warninglist {}".format(misp_warninglist.name)

# A unique reference for the TheHive
thehive_alert_reference = "MISP retroscan warninglist ID - {} - {} - {}".format(misp_warninglist.name, misp_warninglist.id, str(uuid.uuid4()))

# Alert type in TheHive
thehive_alert_type = "MISP Playbook alert"

# TLP:Amber for TheHive
thehive_tlp = 2

# PAP:GREEN for TheHive
thehive_pap = 1

In [16]:
# Code block to send an alert to TheHive
# We use the Python requests library
thehive_headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {thehive_key}'}
thehive_url_create = "{}/api/v1/alert".format(thehive_url)

thehive_observables = []

for value in affected_indicators:
    attribute_type = False
    for expr in regular_expressions:
        if re.match(r"{}".format(regular_expressions[expr]), value):
            attribute_type = expr
            break
    if attribute_type:
        if attribute_type == "ip-src" or attribute_type == "ip-dst":
            dataType = "ip"
        elif attribute_type == "md5" or attribute_type == "sha256" or attribute_type == "sha1":
            dataType = "hash"
        else:
            dataType = attribute_type

        if dataType:
            thehive_observables.append({"dataType": dataType, "data": value, "pap": thehive_pap, "tlp": thehive_tlp})
            
thehive_alert = {"title": thehive_alert_title, 
                 "description": intro,
                 "summary": summary[0:1048576],
                 "type": thehive_alert_type, 
                 "source": "playbook", 
                 "sourceRef": thehive_alert_reference, 
                 "tlp": thehive_tlp, "pap": thehive_pap,
                 "observables": thehive_observables}

result = requests.post(thehive_url_create, headers=thehive_headers, data=json.dumps(thehive_alert))
if result.status_code == 201 and result.json()['status'] == 'New':
    thehive_alert_id = result.json()['_id']
    print('The TheHive \033[92malert {} is added'.format(thehive_alert_id))
else:
    print('\033[91mFailed\033[90m to add TheHive alert')
    print(result.text)

The TheHive alert ~41582720 is added


## EN:4 End of the playbook 

In [17]:
print("\033[92m End of the playbook")


 End of the playbook


## External references <a name="extreferences"></a>

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [TheHive](https://thehive-project.org/)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
pyfaup
chardet
PrettyTable
ipywidgets
```

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```